In [2]:
#from gensim import utils
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
%matplotlib inline

In [19]:
train_data = pd.read_csv('data/IMDB_review_train.csv')
test_data = pd.read_csv('data/IMDB_review_test.csv')
train_texts = list(train_data.text.values)
train_labels = list(train_data.sentiment.values)
test_texts = list(test_data.text.values)
test_labels = list(test_data.sentiment.values)

## Text to Numbers

In [19]:
#from tools.text_preprocess import text_clean
#tc = text_clean(train_texts)
#train_processed = tc.proceed()
#tc = text_clean(test_texts)
#test_processed = tc.proceed()

In [8]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/ubuntu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
train_data = pd.read_csv('data/intermediate_data/train_processed.csv')
test_data = pd.read_csv('data/intermediate_data/test_processed.csv')
train_processed = list(train_data.text.values)
train_labels = list(train_data.sentiment.values)
test_processed = list(test_data.text.values)
test_labels = list(test_data.sentiment.values)

In [9]:
from tools.text_hier_split import sent2words
from tools.token_idx_map import token2idx
sw_train = sent2words(train_processed)
sw_test = sent2words(test_processed)
train_sent_words = sw_train.proceed()
test_sent_words = sw_test.proceed()
ti = token2idx(train_sent_words)
train_sent_idx = ti.proceed()
test_sent_idx = ti.map_text_idx(test_sent_words, ignore_sent=True)

Start mapping words to IDs....
Processing Finished! Timing:  4.309


In [10]:
n_words = len(ti.get_vocab())
print('Total words: %d' % n_words)

Total words: 20001


In [11]:
from tools.sample_generator import generate_samples
MAX_DOCUMENT_LENGTH = 800
gs_train = generate_samples(train_sent_idx, train_labels, MAX_DOCUMENT_LENGTH)
gs_test = generate_samples(test_sent_idx, test_labels, MAX_DOCUMENT_LENGTH)

## Settings

In [12]:
class trainConfig:
    vocab_size = n_words
    max_doc_len = MAX_DOCUMENT_LENGTH
    label_size = 2
    embed_size = 100
    hidden_size = 250
    batch_size = 64
    layer_size = 2
    
class testConfig:
    vocab_size = n_words
    max_doc_len = MAX_DOCUMENT_LENGTH
    label_size = 2
    embed_size = 100
    hidden_size = 250
    batch_size = 64
    layer_size = 2
    
class singleConfig:
    vocab_size = n_words
    max_doc_len = MAX_DOCUMENT_LENGTH
    label_size = 2
    embed_size = 100
    hidden_size = 250#hidden size for hidden state of rnn
    batch_size = 1
    layer_size = 2

In [13]:
train_chunk_num = int(len(train_processed)/trainConfig.batch_size)
test_chunk_num = int(len(test_processed)/trainConfig.batch_size)
remain_num = len(test_processed) - trainConfig.batch_size*test_chunk_num
remain_num

40

## Attention RNN

In [14]:
from models.Bidirectional_RNN_Attention import AttRNN_Model

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [15]:
import tensorflow as tf
graph_attrnn = tf.Graph()
#Create models for training and testing data
#Create models for training and testing data
with graph_attrnn.as_default():
    initializer = tf.random_uniform_initializer(-0.02, 0.02)
    with tf.name_scope('train'):
        #Set different models for different buckets
        with tf.variable_scope("Model", reuse=None, initializer=initializer):
            train_model = AttRNN_Model(trainConfig)
            saver=tf.train.Saver()
    with tf.name_scope('test'):
        #Set different models for different buckets
        with tf.variable_scope("Model", reuse=True, initializer=initializer):
            test_model = AttRNN_Model(testConfig, False)
            single_model = AttRNN_Model(singleConfig, False)

Model Initialized!
Model Initialized!
Model Initialized!


In [16]:
gs_train = generate_samples(train_sent_idx, train_labels, MAX_DOCUMENT_LENGTH)
gs_test = generate_samples(test_sent_idx, test_labels, MAX_DOCUMENT_LENGTH)

In [17]:
import time, os
epochs = 2
#train_chunk_num = 10
file = "ckpt_attrnn/attrnn.ckpt"
with tf.Session(graph=graph_attrnn) as sess:
    #Initialize parameters
    init = tf.global_variables_initializer()
    if not os.path.exists("ckpt_attrnn"):
        os.mkdir('ckpt_attrnn')
    if os.path.exists("ckpt_attrnn/attrnn.ckpt.index"):
        saver.restore(sess, file)
    else:
         sess.run(init)
    start_time = time.time()
    for m in range(epochs):
        for i in range(train_chunk_num):
            #sess.run(tf.assign(learning_rate, 0.002*((0.98)**m)))
            x, y, lengths, _ = gs_train.generate_batch(trainConfig.batch_size)
            feed_dict = {train_model.x:x, train_model.y:y, train_model.lengths:lengths}
            l, _ = sess.run([train_model.cost, train_model.optimize], feed_dict=feed_dict)
            if i%100 == 0:
                print('Loss:', round(l, 4))
        end_time = time.time()
        print('Epoch', m, 'time:{:.2f}'.format(end_time - start_time))
        
    saver.save(sess, file)

Loss: 0.7837
Loss: 0.5488
Loss: 0.3226
Loss: 0.3636
Epoch 0 time:620.27
Loss: 0.2043
Loss: 0.3661
Loss: 0.3672
Loss: 0.2772
Epoch 1 time:1224.11


In [18]:
#Calculate Testing Accuracy
with tf.Session(graph=graph_attrnn) as sess:
    print('Testing...')
    count = 0
    #saver = tf.train.import_meta_graph('ckpt_cnn/cnn.ckpt.meta')
    saver.restore(sess,tf.train.latest_checkpoint('ckpt_attrnn/'))
    print('Parameters restored')
    start_time = time.time()
    test_gs = generate_samples(np.array(test_processed), np.array(test_labels), False)
    for _ in range(test_chunk_num):
        #Traverse each data
        x, y, lengths, _ = gs_test.generate_batch(testConfig.batch_size)
        feed_dict = {test_model.x:x, test_model.y:y, test_model.lengths:lengths}
        n = sess.run(test_model.correct_num, feed_dict=feed_dict)
        count += np.sum(n)
    for _ in range(remain_num):
        #Traverse each data
        x, y, lengths, _ = gs_test.generate_batch(1)
        feed_dict = {single_model.x:x, single_model.y:y, 
                     single_model.lengths:lengths}
        n = sess.run(single_model.correct_num, feed_dict=feed_dict)
        count += np.sum(n)
    end_time = time.time()
    print('Testing Time:{:.2f}'.format(end_time - start_time))
    print(count*1.0/len(test_processed)) 

Testing...
INFO:tensorflow:Restoring parameters from ckpt_attrnn/attrnn.ckpt
Parameters restored
Testing Time:251.52
0.87556


Attention model works really well.

In [79]:
import time, os
epochs = 1
#train_chunk_num = 10
file = "ckpt/hirnn.ckpt"
with tf.Session(graph=graph_attrnn) as sess:
    #Initialize parameters
    init = tf.global_variables_initializer()
    sess.run(init)
    if os.path.exists("ckpt/hirnn.ckpt.index"):
        saver.restore(sess, file)
    start_time = time.time()
    for m in range(epochs):
        for i in range(train_chunk_num):
            #sess.run(tf.assign(learning_rate, 0.002*((0.98)**m)))
            x, y, lengths, _ = gs_train.generate_batch(trainConfig.batch_size)
            feed_dict = {train_data:x, train_label:y, train_lengths:lengths}
            l, _ = sess.run([train_model.cost, train_model.optimize], feed_dict=feed_dict)
            if i%100 == 0:
                print('Loss:', round(l, 4))
        end_time = time.time()
        print('Epoch', m, 'time:{:.2f}'.format(end_time - start_time))
        start_time = end_time
    saver.save(sess,'ckpt/hirnn.ckpt')
    #Calculate Testing Accuracy
    print('Testing...')
    count = 0
    gs_test = generate_samples(test_sent_idx, test_labels, MAX_DOCUMENT_LENGTH)
    for _ in range(test_chunk_num):
        #Traverse each data
        x, y, lengths, _ = gs_test.generate_batch(testConfig.batch_size, False)
        feed_dict = {test_data:x, test_label:y, test_lengths:lengths}
        n = sess.run(test_model.correct_num, feed_dict=feed_dict)
        count += np.sum(n)
    for _ in range(remain_num):
        #Traverse each data
        x, y, lengths, _ = gs_test.generate_batch(1, False)
        feed_dict = {single_data:x, single_label:y, single_lengths:lengths}
        n = sess.run(single_model.correct_num, feed_dict=feed_dict)
        count += np.sum(n)
    end_time = time.time()
    print('Testing Time:{:.2f}'.format(end_time - start_time))
    print(count*1.0/len(test_texts))  

Loss: 0.6934
Loss: 0.6142
Loss: 0.3579
Loss: 0.2406
Epoch 0 time:1072.45
Testing...
Test Samples come to an end!
Testing Time:274.85
0.86724
